# Europe Fuel Prices
## Data treatment

## Data

http://www.fuel-prices-europe.info/

In [1]:
FULL_DATA = 'data/europe-fuel-prices.csv'

## Global imports

In [2]:
%cd ..
import config
%cd europe-fuel-prices

C:\Users\USER\Dropbox\Projectos\data-viz
C:\Users\USER\Dropbox\Projectos\data-viz\europe-fuel-prices


## Load data

In [3]:
import pandas as pd
import numpy as np


# Load from CSV
df = pd.read_csv(FULL_DATA, delimiter=';')

# Clean column names
df.columns = [c.lower().replace(' ', '_') for c in df.columns]

df.head(5)

,country,eu,europe,unleaded_95_eur,unleaded_95_price,diesel_eur,diesel_price,lpg_eur,lpg_price,lpg_nozzle,lpg_stations,record_date
0,Albania,No,Yes,0.000,ALL 172.00,0.000,ALL 169.00,0.000,ALL 72.00,Dish,70,"January 2, 2020"
1,Algeria,No,No,0.000,DZD 22.60,0.000,DZD 13.70,0.000,DZD 9.00,Unknown,Unknown,"January 2, 2020"
2,Andorra,No,Yes,1.164,NaN,1.054,NaN,,NaN,Unknown,1,"January 2, 2020"
3,Armenia,No,Yes,0.000,AMD 510.00,0.000,AMD 480.00,,NaN,Unknown,Unknown,"January 2, 2020"
4,Austria,Yes,Yes,1.169,NaN,1.155,NaN,0.779,NaN,Dish,26,"January 2, 2020"


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56 entries, 0 to 55
Data columns (total 12 columns):
country              56 non-null object
eu                   56 non-null object
europe               56 non-null object
unleaded_95_eur      56 non-null float64
unleaded_95_price    34 non-null object
diesel_eur           56 non-null float64
diesel_price         34 non-null object
lpg_eur              56 non-null object
lpg_price            22 non-null object
lpg_nozzle           56 non-null object
lpg_stations         56 non-null object
record_date          56 non-null object
dtypes: float64(2), object(10)
memory usage: 5.4+ KB


## Data treatment

In [5]:
# Get fuel prices in local currencies
unleaded_95 = df['unleaded_95_price'].str.split(' ', n=1, expand=True) 
diesel = df['diesel_price'].str.split(' ', n=1, expand=True)
lpg = df['lpg_price'].str.split(' ', n=1, expand=True)

# Where there are no 'local' currency input EUR
unleaded_95[0].fillna('EUR', inplace=True)

# Create new columns
df['currency'] = unleaded_95[0]
df['unleaded_95_local_price'] = unleaded_95[1]
df['diesel_local_price'] = diesel[1]
df['lpg_local_price'] = lpg[1]

# Remove processed rows
df.drop(columns=['unleaded_95_price', 'diesel_price', 'lpg_price'], inplace=True)

df.head(5)

,country,eu,europe,unleaded_95_eur,diesel_eur,lpg_eur,lpg_nozzle,lpg_stations,record_date,currency,unleaded_95_local_price,diesel_local_price,lpg_local_price
0,Albania,No,Yes,0.000,0.000,0.000,Dish,70,"January 2, 2020",ALL,172.00,169.00,72.00
1,Algeria,No,No,0.000,0.000,0.000,Unknown,Unknown,"January 2, 2020",DZD,22.60,13.70,9.00
2,Andorra,No,Yes,1.164,1.054,,Unknown,1,"January 2, 2020",EUR,NaN,NaN,NaN
3,Armenia,No,Yes,0.000,0.000,,Unknown,Unknown,"January 2, 2020",AMD,510.00,480.00,NaN
4,Austria,Yes,Yes,1.169,1.155,0.779,Dish,26,"January 2, 2020",EUR,NaN,NaN,NaN


In [6]:
def fill_prices(row, local_price, eur):
    """Where the local price is 0 or missing, input the price in EUR"""
    if row[local_price] is np.nan or row[local_price] == 0:
        return row[eur]
    else:
        return row[local_price]

In [7]:
df['unleaded_95_local_price'] = df.apply(lambda row : fill_prices(row, 'unleaded_95_local_price', 'unleaded_95_eur'), axis=1)
df['diesel_local_price'] = df.apply(lambda row : fill_prices(row, 'diesel_local_price', 'diesel_eur'), axis=1)
df['lpg_local_price'] = df.apply(lambda row : fill_prices(row, 'lpg_local_price', 'lpg_eur'), axis=1)

df['unleaded_95_local_price'] = pd.to_numeric(df['unleaded_95_local_price'], errors='coerce')
df['diesel_local_price'] = pd.to_numeric(df['diesel_local_price'], errors='coerce')
df['lpg_local_price'] = pd.to_numeric(df['lpg_local_price'], errors='coerce')

df['unleaded_95_eur'] = pd.to_numeric(df['unleaded_95_eur'], errors='coerce')
df['diesel_eur'] = pd.to_numeric(df['diesel_eur'], errors='coerce')
df['lpg_eur'] = pd.to_numeric(df['lpg_eur'], errors='coerce')

df.head(5)

,country,eu,europe,unleaded_95_eur,diesel_eur,lpg_eur,lpg_nozzle,lpg_stations,record_date,currency,unleaded_95_local_price,diesel_local_price,lpg_local_price
0,Albania,No,Yes,0.000,0.000,0.000,Dish,70,"January 2, 2020",ALL,172.000,169.000,72.000
1,Algeria,No,No,0.000,0.000,0.000,Unknown,Unknown,"January 2, 2020",DZD,22.600,13.700,9.000
2,Andorra,No,Yes,1.164,1.054,NaN,Unknown,1,"January 2, 2020",EUR,1.164,1.054,NaN
3,Armenia,No,Yes,0.000,0.000,NaN,Unknown,Unknown,"January 2, 2020",AMD,510.000,480.000,NaN
4,Austria,Yes,Yes,1.169,1.155,0.779,Dish,26,"January 2, 2020",EUR,1.169,1.155,0.779


In [8]:
# Get currencies
currency_list = df['currency'].unique()
currency_query_string = ','.join(currency_list)

# Get exchange rates from API
import requests
import config

params={
    'access_key': config.FIXER_API_KEY,
    'symbols': currency_query_string,
    'base': 'EUR'
}
r = requests.get('http://data.fixer.io/api/latest', params=params)

if r:
    print(f"SUCCESS (Status: {r.status_code})")
    json_response = r.json()
    print(json_response)
    
else:
    print(f"ERROR (Status: {r.status_code})")

# Create exchange rate dict
rates = json_response['rates']

SUCCESS (Status: 200)
{'success': True, 'timestamp': 1578065706, 'base': 'EUR', 'date': '2020-01-03', 'rates': {'ALL': 121.939081, 'DZD': 133.159421, 'EUR': 1, 'AMD': 534.339397, 'BYN': 2.354463, 'BAM': 1.961297, 'BGN': 1.955715, 'HRK': 7.445873, 'CZK': 25.352163, 'DKK': 7.473159, 'EGP': 17.905441, 'GEL': 3.192234, 'HUF': 330.169466, 'ISK': 136.94735, 'ILS': 3.881744, 'JOD': 0.791683, 'KWD': 0.338337, 'LBP': 1696.530465, 'LYD': 1.565557, 'MKD': 61.494468, 'MDL': 19.252964, 'MAD': 10.723373, 'NOK': 9.83707, 'PLN': 4.243027, 'RON': 4.778113, 'RUB': 69.166127, 'SAR': 4.188128, 'RSD': 117.502244, 'SEK': 10.474021, 'CHF': 1.084682, 'TND': 3.110728, 'TRY': 6.668082, 'USD': 1.116146, 'UAH': 26.414928, 'GBP': 0.852936}}


In [9]:
def convert_to_eur(row, local_price, eur):
    """If the EUR price is 0 input it with the value converted from the local currency"""
    if row[eur] == 0:
        return row[local_price] / rates[row['currency']]
    else:
        return row[eur]

In [10]:
df['unleaded_95_eur'] = df.apply(lambda row : convert_to_eur(row, 'unleaded_95_local_price', 'unleaded_95_eur'), axis=1)
df['diesel_eur'] = df.apply(lambda row : convert_to_eur(row, 'diesel_local_price', 'diesel_eur'), axis=1)
df['lpg_eur'] = df.apply(lambda row : convert_to_eur(row, 'lpg_local_price', 'lpg_eur'), axis=1)
df.head(5)

,country,eu,europe,unleaded_95_eur,diesel_eur,lpg_eur,lpg_nozzle,lpg_stations,record_date,currency,unleaded_95_local_price,diesel_local_price,lpg_local_price
0,Albania,No,Yes,1.410540,1.385938,0.590459,Dish,70,"January 2, 2020",ALL,172.000,169.000,72.000
1,Algeria,No,No,0.169721,0.102884,0.067588,Unknown,Unknown,"January 2, 2020",DZD,22.600,13.700,9.000
2,Andorra,No,Yes,1.164000,1.054000,NaN,Unknown,1,"January 2, 2020",EUR,1.164,1.054,NaN
3,Armenia,No,Yes,0.954450,0.898305,NaN,Unknown,Unknown,"January 2, 2020",AMD,510.000,480.000,NaN
4,Austria,Yes,Yes,1.169000,1.155000,0.779000,Dish,26,"January 2, 2020",EUR,1.169,1.155,0.779


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56 entries, 0 to 55
Data columns (total 13 columns):
country                    56 non-null object
eu                         56 non-null object
europe                     56 non-null object
unleaded_95_eur            56 non-null float64
diesel_eur                 56 non-null float64
lpg_eur                    40 non-null float64
lpg_nozzle                 56 non-null object
lpg_stations               56 non-null object
record_date                56 non-null object
currency                   56 non-null object
unleaded_95_local_price    56 non-null float64
diesel_local_price         56 non-null float64
lpg_local_price            40 non-null float64
dtypes: float64(6), object(7)
memory usage: 5.8+ KB


In [12]:
df.describe()

,unleaded_95_eur,diesel_eur,lpg_eur,unleaded_95_local_price,diesel_local_price,lpg_local_price
count,56.000000,56.000000,40.000000,56.000000,56.000000,40.000000
mean,1.159790,1.111671,0.605885,76.105179,63.367911,49.315250
std,0.416057,0.422101,0.177264,335.955182,248.337679,229.094452
min,0.127750,0.095813,0.067588,0.065000,0.110000,0.450000
25%,1.041851,0.897073,0.509795,1.275750,1.219500,0.625250
50%,1.216145,1.186693,0.586729,1.624000,1.532000,0.876500
75%,1.429338,1.392703,0.727197,13.487500,11.967500,9.000000
max,1.804000,2.107300,0.879000,2450.000000,1760.000000,1440.000000


In [13]:
# Save to CSV
df.to_csv('data/europe-fuel-prices-final.csv', sep=';', index=False)